In [1]:
import pandas as pd

# 파일 경로 설정 (본인의 환경에 맞게 수정하세요)
file1_path = '[0203최종]merged_행정구역.xlsx'
file2_path = r"C:\Users\bag43\Desktop\예시 데이터셋\전처리작업용\찐찐최종\거주인구.xlsx"
output_file_path = 'merged_final_result.xlsx'

# 데이터 불러오기
data1 = pd.read_excel(file1_path)
data2 = pd.read_excel(file2_path)

# 1. 데이터 준비: 열 이름 정리
data2 = data2.rename(columns={'행정구역(시군구)별': 'district'})

# 2. 병합 작업
# 출발지 매핑
o_merged = data1.merge(
    data2, left_on='O_district', right_on='district', how='left', suffixes=('', '_출발지')
)

# 도착지 매핑
final_data = o_merged.merge(
    data2, left_on='D_district', right_on='district', how='left', suffixes=('', '_도착지')
)

# 3. 필요한 열 이름 변경
for col in data2.columns[1:]:
    final_data.rename(columns={col: f"출발지_{col}"}, inplace=True)
    final_data.rename(columns={f"{col}_도착지": f"도착지_{col}"}, inplace=True)

# 4. 중복 열 제거
final_data = final_data.drop(columns=['district', 'district_도착지'])

# 5. 결과 데이터 저장
final_data.to_excel(output_file_path, index=False)

# 6. 행 및 열 개수 확인
print("원본 데이터 행 개수:", data1.shape[0])
print("최종 데이터 행 개수:", final_data.shape[0])
print("원본 데이터 열 개수:", data1.shape[1])
print("최종 데이터 열 개수:", final_data.shape[1])


원본 데이터 행 개수: 129979
최종 데이터 행 개수: 129979
원본 데이터 열 개수: 28
최종 데이터 열 개수: 72


In [1]:
import pandas as pd

# 1. 파일 경로 설정
file1_path = '[0203최종]merged_행정구역.xlsx'
file2_path = r"C:\Users\bag43\Desktop\자료실\자료실\외부 활동\▲동아리\DSL\2025-1\EDA\(0202최종)regiondata_02021729.xlsx"
output_file_path = 'merged_final_result.xlsx'

# 2. file1 데이터 불러오기
data1 = pd.read_excel(file1_path)

# 3. file2의 모든 시트 이름 읽어오기
excel_file = pd.ExcelFile(file2_path)
sheet_names = excel_file.sheet_names

# 4. 각 시트별로 병합 작업 수행
merged_results = {}  # 시트별 결과 저장용 딕셔너리

for sheet in sheet_names:
    # 4-1. file2의 특정 시트를 읽어오기
    data2 = pd.read_excel(file2_path, sheet_name=sheet)
    
    # 4-2. 열 이름 정리: 'end_district'를 'district'로 변경
    data2 = data2.rename(columns={'end_district': 'district'})
    
    # (선택사항) - 만약 월별 데이터에 포함된 쉼표 때문에 숫자 인식에 문제가 있다면,
    # 아래와 같이 처리하여 숫자형으로 변환할 수 있습니다.
    # cols_to_convert = data2.columns.drop('district')
    # data2[cols_to_convert] = data2[cols_to_convert].replace({',': ''}, regex=True).astype(float)
    
    # 4-3. 출발지 매핑: data1과 data2를 O_district 기준으로 병합
    o_merged = data1.merge(
        data2, left_on='O_district', right_on='district', how='left', suffixes=('', '_출발지')
    )
    
    # 4-4. 도착지 매핑: 위 결과와 data2를 D_district 기준으로 병합
    merged = o_merged.merge(
        data2, left_on='D_district', right_on='district', how='left', suffixes=('', '_도착지')
    )
    
    # 4-5. file2의 merge key가 아닌 나머지 열들에 대해 접두사 추가 작업  
    # file2의 첫 번째 열은 merge key('district')이므로 제외하고 나머지 열들에 대해 처리합니다.
    for col in data2.columns:
        if col == 'district':
            continue
        # 출발지 데이터에 대한 열 이름 변경
        if col in merged.columns:
            merged.rename(columns={col: f"출발지_{col}"}, inplace=True)
        # 도착지 데이터에 대한 열 이름 변경 (merge 시 suffix '_도착지'가 붙은 열)
        col_d = f"{col}_도착지"
        if col_d in merged.columns:
            merged.rename(columns={col_d: f"도착지_{col}"}, inplace=True)
    
    # 4-6. 중복된 merge key 열 제거 (첫번째 병합 시 생성된 'district'와 도착지 병합 시의 'district')
    merged.drop(columns=['district', 'district_도착지'], inplace=True, errors='ignore')
    
    # 4-7. 시트별 결과 저장 (key는 시트 이름)
    merged_results[sheet] = merged

# 5. 결과 데이터를 하나의 엑셀 파일의 개별 시트로 저장
with pd.ExcelWriter(output_file_path) as writer:
    for sheet, df in merged_results.items():
        df.to_excel(writer, sheet_name=sheet, index=False)

# 6. 최종 결과 출력 (file1의 행 개수와 각 시트별 행/열 개수)
print("file1 원본 데이터 행 개수:", data1.shape[0])
for sheet, df in merged_results.items():
    print(f"시트 [{sheet}] - 최종 데이터 행 개수: {df.shape[0]}")
    print(f"시트 [{sheet}] - 최종 데이터 열 개수: {df.shape[1]}")


file1 원본 데이터 행 개수: 129979
시트 [resident] - 최종 데이터 행 개수: 129979
시트 [resident] - 최종 데이터 열 개수: 72
시트 [medical] - 최종 데이터 행 개수: 129979
시트 [medical] - 최종 데이터 열 개수: 72
시트 [housing] - 최종 데이터 행 개수: 129979
시트 [housing] - 최종 데이터 열 개수: 72
시트 [movingpop] - 최종 데이터 행 개수: 129979
시트 [movingpop] - 최종 데이터 열 개수: 72
시트 [net_movingpop] - 최종 데이터 행 개수: 129979
시트 [net_movingpop] - 최종 데이터 열 개수: 72


In [1]:
import pandas as pd

# --- 0. 행정구역 정규화 함수 정의 ---
def normalize_region(region):
    """
    행정구역명을 정규화하는 함수.
    - 특정 문자열(삭제 조건)이 포함되면 None 반환하여 이후 행 삭제
    - 지정된 매핑 규칙에 따라 부분 문자열 치환을 수행함.
      (포함 여부를 기준으로 치환하므로 '인천시 중구'도 '인천광역시 중구'로 처리)
    """
    if pd.isna(region):
        return region
    r = str(region).strip()
    
    # 삭제 조건: 만약 아래 단어가 포함되어 있으면 None 반환
    drop_list = ['충남', '충청남도', '강원도', '춘천', '춘춘']
    for drop in drop_list:
        if drop in r:
            return None
    
    # 매핑 규칙 (더 구체적인 조건부터 체크)
    # 2번: '인천시 남구'가 포함되면 '인천광역시 미추홀구'로 치환
    if "인천시 남구" in r:
        r = r.replace("인천시 남구", "인천광역시 미추홀구")
    # 1번: '인천시'가 포함되면 '인천광역시'로 치환  
    # (단, 이미 '인천광역시'가 있는 경우는 제외)
    elif "인천시" in r:
        r = r.replace("인천시", "인천광역시")
    
    # 6번: '시승시'가 포함되면 '시흥시'로 치환
    if "시승시" in r:
        r = r.replace("시승시", "시흥시")
    
    # 7번: '서울 강서구'가 포함되면 '서울특별시 강서구'로 치환
    if "서울 강서구" in r:
        r = r.replace("서울 강서구", "서울특별시 강서구")
    
    return r

# --- 1. 파일 경로 설정 ---
file1_path = '[0203최종]merged_행정구역.xlsx'
file2_path = r"C:\Users\bag43\Desktop\자료실\자료실\외부 활동\▲동아리\DSL\2025-1\EDA\(0203최종)regiondata_1950.xlsx"
output_file_path = '(0203최종_2)merged_final_result.xlsx'

# --- 2. file1 데이터 불러오기 ---
data1 = pd.read_excel(file1_path)

# --- 2-1. file1의 출발지, 도착지 행정구역 정규화 및 제외 ---
for col in ['O_district', 'D_district']:
    data1[col] = data1[col].apply(normalize_region)
# 제외 조건에 해당하는 행 삭제 (두 컬럼 중 하나라도 None이면 삭제)
data1 = data1.dropna(subset=['O_district', 'D_district']).reset_index(drop=True)

# --- 3. file2의 모든 시트 이름 읽어오기 ---
excel_file = pd.ExcelFile(file2_path)
sheet_names = excel_file.sheet_names

# --- 4. 각 시트별로 병합 작업 수행 ---
merged_results = {}  # 시트별 결과 저장용 딕셔너리

for sheet in sheet_names:
    # 4-1. file2의 특정 시트를 읽어오기
    data2 = pd.read_excel(file2_path, sheet_name=sheet)
    
    # 4-2. 'end_district'를 'district'로 변경
    data2 = data2.rename(columns={'end_district': 'district'})
    
    # --- file2의 district 컬럼 정규화 및 제외 ---
    data2['district'] = data2['district'].apply(normalize_region)
    data2 = data2.dropna(subset=['district']).reset_index(drop=True)
    
    # (선택사항) - 쉼표로 인한 숫자 인식 문제 처리 (필요 시 주석 해제)
    # cols_to_convert = data2.columns.drop('district')
    # data2[cols_to_convert] = data2[cols_to_convert].replace({',': ''}, regex=True).astype(float)
    
    # --- file1과 file2의 키 값이 정규화되었으므로 그대로 병합 진행 ---
    # 4-3. 출발지 매핑: data1과 data2를 O_district 기준으로 병합
    o_merged = data1.merge(
        data2, left_on='O_district', right_on='district', how='left', suffixes=('', '_출발지')
    )
    
    # 4-4. 도착지 매핑: 위 결과와 data2를 D_district 기준으로 병합
    merged = o_merged.merge(
        data2, left_on='D_district', right_on='district', how='left', suffixes=('', '_도착지')
    )
    
    # 4-5. file2의 merge key('district')가 아닌 나머지 열들에 대해 접두사 추가 작업  
    # file2의 첫 번째 열은 merge key('district')이므로 제외하고 나머지 열들에 대해 처리합니다.
    for col in data2.columns:
        if col == 'district':
            continue
        # 출발지 데이터에 대한 열 이름 변경 (이미 merge 시에는 접두사 없음)
        if col in merged.columns:
            merged.rename(columns={col: f"출발지_{col}"}, inplace=True)
        # 도착지 데이터에 대한 열 이름 변경 (merge 시 suffix '_도착지'가 붙은 열)
        col_d = f"{col}_도착지"
        if col_d in merged.columns:
            merged.rename(columns={col_d: f"도착지_{col}"}, inplace=True)
    
    # 4-6. 중복된 merge key 열 제거 (첫번째 병합 시 생성된 'district'와 도착지 병합 시의 'district')
    merged.drop(columns=['district', 'district_도착지'], inplace=True, errors='ignore')
    
    # 4-7. 시트별 결과 저장 (key는 시트 이름)
    merged_results[sheet] = merged

# --- 5. 결과 데이터를 하나의 엑셀 파일의 개별 시트로 저장 ---
with pd.ExcelWriter(output_file_path) as writer:
    for sheet, df in merged_results.items():
        df.to_excel(writer, sheet_name=sheet, index=False)

# --- 6. 최종 결과 출력 (file1의 행 개수와 각 시트별 행/열 개수) ---
print("file1 원본 데이터 행 개수:", data1.shape[0])
for sheet, df in merged_results.items():
    print(f"시트 [{sheet}] - 최종 데이터 행 개수: {df.shape[0]}")
    print(f"시트 [{sheet}] - 최종 데이터 열 개수: {df.shape[1]}")


file1 원본 데이터 행 개수: 123750
시트 [resident] - 최종 데이터 행 개수: 123750
시트 [resident] - 최종 데이터 열 개수: 72
시트 [medical] - 최종 데이터 행 개수: 123750
시트 [medical] - 최종 데이터 열 개수: 72
시트 [housing] - 최종 데이터 행 개수: 123750
시트 [housing] - 최종 데이터 열 개수: 72
시트 [movingpop] - 최종 데이터 행 개수: 123750
시트 [movingpop] - 최종 데이터 열 개수: 72
시트 [net_movingpop] - 최종 데이터 행 개수: 123750
시트 [net_movingpop] - 최종 데이터 열 개수: 72
